# Subject Data PreProcessing
* find subject data
* concatenate the inter-session data
* concatenate the intra-session data

# TO DO
* Adapt the cleaning cell to also consider empty data points and mark them as noData in the DataFrame
* Write gaze definition cell by counting identical collider appearances while 21 consecutive hits are then defined as a gaze caused by an actual fixation. 


## Configuration

In [1]:
# General configuration
import os

# data_directory: str
#     Path to a directory to store data.
data_directory = '.'

# install_missing_packages: bool
#     A flag indicating if missing packages should be automatically installed
install_missing_packages = True

# use_conda: bool
#     A flag indicating if conda should be used for software installation.
#     If False, pip will be used. The default is to use conda if jupyter
#     is run in a conda environment.
use_conda = 'CONDA_EXE' in os.environ

## Checking for missing packages

In [2]:
import importlib

def check_package(package, pip_pkg: str = None, conda_pkg: str = None):
    """Check if a given package is installed. If missing install
    it (if global flag `install_missing_packages` is True) either with
    pip or with conda (depending on `use_conda`).
    """
    if importlib.util.find_spec(package) is not None:
        return  # ok, package is already installed

    if not install_missing_packages:
        raise RuntimeError(f"{package} is not installed!")

    if use_conda:
        import conda.cli
        conda.cli.main('conda', 'install',  '-y', conda_pkg or package)
    else:
        import subprocess
        import sys            
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_pkg or package])
        
# This is to exit cells without error tracebacks (cosmetic purpose)
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Creating the required environment (skip if already done)

Running the following cell will create a file graphs.yml that can be used to setup a conda environment containing the required packages. If you already downloaded the file from my GitHub, skip the next cell and create the env directly from it.

In [3]:
%%writefile graphs.yml
name: graphs
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.6
  - jupyter
  - imageio
  - imageio-ffmpeg
  - matplotlib
  - scikit-image
  - opencv
  - networkx
  - pandas
  - statsmodels

Writing graphs.yml


### Environment Creation
To create the environment, open the terminal, go to the directory where you stored the graphs.yml file (the directory of the notebook) and type
conda env create -f graphs.yml
After running this command you have to activate the environment (Linux/MacOS: conda activate graphs, Windows: activate graphs) and then reopen the notebook in that environment.

## Main Part

### Imports and directory information

In [28]:
import os
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nX
import glob
import scipy.cluster.vq as clusters
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import normalize
from pandas.plotting import autocorrelation_plot as AC_plot 
#from statsmodels.graphics import tsaplots
#from statsmodels.tsa.stattools import acf
from skimage.filters import gaussian
from mpl_toolkits.mplot3d import Axes3D 
import time
import random



In [127]:
OG_DATA_PATH = './'

condition = 'Single' # Single, Dyadic, SingleC
DATA_PATH = './Data {}/'.format(condition)

PROCESSED_DATA_PATH = './Results/' + condition + '/'

### Extracting all subject IDs from the data folder

In [128]:
# Getting the Folder without hidden files in ascending order 
DATA_FOLDER = sorted([f for f in os.listdir(DATA_PATH) if not f.startswith('.')], key=str.lower)

subIDs = []
for sub in DATA_FOLDER:
    if sub[0].isdigit():
        subIDs.append(int(sub[0:4]))
    else:
        pass
subIDs = np.unique(subIDs)


#subIDs = [1023] # remove to do for all subIDs
print(subIDs)

[1004 1005 1008 1010 1011 1013 1017 1018 1019 1021 1022 1023 1054 1055
 1056 1057 1058 1068 1069 1072 1073 1074 1075 1077 1079 1080]


# Combining Pathfinding Experiment Files per Subject

In [24]:
EXPT_FILE_ATTRS = ['ParticipantID', 'PathsReversed', 'IsDyadic', 'IsLeader'];


def map_dict(a_dictionary):
    a_subset = {key: value for key, value in a_dictionary.items() if key in EYE_FILE_ATTRS};
    return a_subset;

# For each subject in subject ID folder, combine all experiment files
# and save as a unified final experiment file in Result folder
for sub in subIDs:
    # Get all files for subject
    SUB_FILES = list(filter(lambda x: str(sub)+"_" in x, DATA_FOLDER));
    # Sort files into experiment files and eyetracking files
    EYETRACKING_FILES = list(filter(lambda x: 'EyeTracking' in x, SUB_FILES));
    EYETRACKING_FILES.sort();
    
    EXPT_FILES = list(filter(lambda x: condition + 'PathFinding' in x, SUB_FILES));
    EXPT_FILES.sort();
    
    EXPT_DATA = {};
        
    # For each experiment file
    for expt_file in EXPT_FILES:
        
        # Read JSON data
        with open(DATA_PATH + expt_file) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + expt_file + " is not valid!")
                continue;
        
        # If there are more than one experiment files, combine the data of the paths if the 
        # other trial and subject information matches
        if len(EXPT_DATA) > 0:
            print([subject_session[x] for x in EXPT_FILE_ATTRS]);
            print([EXPT_DATA[x] for x in EXPT_FILE_ATTRS]);
            if(not all([subject_session[x] == EXPT_DATA[x] for x in EXPT_FILE_ATTRS])):
                raise Exception("Experiment File Data " + expt_file + " Does Not Match!")
            else:
                EXPT_DATA['Concatenated'] = True;
                EXPT_DATA['TrialData'] += subject_session['TrialData'];
        
        else:
            EXPT_DATA = subject_session;
            EXPT_DATA['Concatenated'] = False;
        
    # Check if correct number of paths present in experiment file
    if len(EXPT_DATA['TrialData']) != 10:
        raise Exception("Incorrent number of paths in file for Subject " + str(sub));
        break;
        
    # Save the final combined file in results to be accessed by eyetracking collection script
    expt_file_name = PROCESSED_DATA_PATH + str(sub) + "_"+condition+"Pathfinding_Final.json";
    with open(expt_file_name, 'w') as fp:
        json.dump(EXPT_DATA, fp);
    EXPT_DATA = {};
    

In [30]:
path_time_limits_A = [300, 305, 234, 332, 268, 394, 152, 383, 332];
path_time_limits_B = path_time_limits_A.copy();
path_time_limits_B.reverse();


# Leader : follower map
dyadic_pair_map = {
    '1021' : '1022',
    '1023' : '1013',
    '1005' : '1055',
    '1074' : '1069',
    '1008' : '1058',
    '1054' : '1004',
    '1011' : '1017',
    '1018' : '1057'
}

dyadic_pair_map_twoway = dyadic_pair_map.copy();

dyadic_pair_map_twoway.update({v:k for (k,v) in dyadic_pair_map.items()});

dyadic_session_ids = {}
for leader in dyadic_pair_map.keys():
    _id = random.randint(1000,9999);
    dyadic_session_ids[leader] = _id;
    dyadic_session_ids[dyadic_pair_map[leader]] = _id;
    
print(dyadic_session_ids);

[332, 383, 152, 394, 268, 332, 234, 305, 300]
{'1021': 3087, '1022': 3087, '1023': 6723, '1013': 6723, '1005': 4175, '1055': 4175, '1074': 3891, '1069': 3891, '1008': 5833, '1058': 5833, '1054': 9758, '1004': 9758, '1011': 9779, '1017': 9779, '1018': 5924, '1057': 5924}


### Combine the eyetracking data
* Loop through all subjects
* extract the session data
* combine the data
* save it

In [71]:
# --------- Preparation ---------

Session_save_bool = False # set to True if you want to save each individual session as csv
Exploration_save_bool = True # set to True if you want to save the complete exploration as csv
subcount = 0

# column name list for dataframe
col_names =  ['PathNumber',
          'timeStampDataPointStart',
          'timeStampDataPointEnd',
          'serverTimeStampDataPointStart',
          'serverTimeStampDataPointEnd',
          'hitObjectColliderName', 
          'ordinalOfHit',
          'BitMask',
          'hitPointOnObject.x',
          'hitPointOnObject.y',
          'hitPointOnObject.z',
          'hitObjectColliderBoundsCenter.x',
          'hitObjectColliderBoundsCenter.y',
          'hitObjectColliderBoundsCenter.z',
            'hmdPosition.x',
             'hmdPosition.y',
             'hmdPosition.z']


# TODO: Add server times to nohitdict
NoHit_dict = {'hitPointOnObject': {'x': 'NaN',
                                   'y': 'NaN',
                                   'z': 'NaN'},
              'hitObjectColliderName': 'NoHit',
              'hitObjectColliderBoundsCenter': {'x': 'NaN',
                                                'y': 'NaN',
                                                'z': 'NaN'},
              'ordinalOfHit': 'NaN',
              'PathNumber': 'NaN',
              'timeStampDataPointStart': 'NaN',
              'timeStampDataPointEnd': 'NaN',
              'serverTimeStampDataPointStart' : 'NaN',
              'serverTimeStampDataPointEnd' : 'NaN',
              'BitMask': 'NaN',
              'hmdPosition': {'x': 'NaN',
                              'y': 'NaN',
                              'z': 'Nan'}
              }

# Remove machine time stamps if single
if 'Single' in condition:
    col_names.pop(3);
    col_names.pop(3);
    NoHit_dict.pop('serverTimeStampDataPointStart');
    NoHit_dict.pop('serverTimeStampDataPointEnd');



# --------------------------- MAIN PART ---------------------------


# --------- first layer - subject loop ---------

for subject in subIDs:
    subcount +=1
    print('Subject ' + str(subject) + ' started - ' + str(subcount) + '/' + str(len(subIDs)))
    
    # Create empty dataframe for later concatenation
    complete_exploration_df = pd.DataFrame(columns = col_names)
    
    # get the data files according to the subject
    subject_folder = sorted([f for f in DATA_FOLDER 
                             if f.startswith(str(subject)+'_EyeTrackingData_')], 
                            key=str.lower) 
    
    # the following works as long as the data name format is as follows:
    # 'subjectID'_Expl_S_'SessionNumber'_ET_'EyeTrackingSessionNumber'_'UnixTimestamp'.json
    folder_files = list()
       
    # loop through the subject folder and save all numbers
    for file in subject_folder:
        folder_files.append(re.findall(r'\d+', file))
    
    if len(folder_files) == 0:
        raise Exception("No eyetracking files in " + condition +" condition for subject " + str(subject))
    # Extract all SubIDs (only one), (and Timestamps)
    SubID, UnixTimestamp1, _ = map(list, zip(*folder_files))
    
    # Get the experiment JSON file for the subject
    sub_expt_data = {}
    sub_expt_file_name = PROCESSED_DATA_PATH + str(subject) + "_"+condition+"Pathfinding_Final.json";
    try:
        with open(sub_expt_file_name, 'r') as fp:
            sub_expt_data = json.load(fp);
    except:
        raise Exception("Could not read experiment file for subject " + str(subject));
    
# --------- second layer - exploration session loop ---------
    s = 0;
    complete_hitpoints_df = pd.DataFrame(columns = col_names)
    previous_path_num = -1.0;
    # loop over separate eye tracking sessions
    for ET_session in subject_folder:
        s+=1
        #print("\tSession " + str(s) + " started")

            # open the JSON file as dictionary
        with open(DATA_PATH + ET_session) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + ET_session + " is not valid!")

        hitpoint_list = list() # create hitpoint list
        
        # check if trial is valid:
        trial_valid = subject_session['trials'][0]['trialIsValid']
        if not trial_valid:
            raise Exception("Excluding " + ET_Session + " because trial is not valid!");
            continue;
        
        sub_id = subject_session['trials'][0]['participantId']
        if str(sub_id) != str(subject):
            raise Exception("Excluding " + ET_Session + " because subject IDs don't match!");
            continue;
        
        # start timestamp of the session 
        start_time = subject_session['trials'][0]['timeTrialMeasurementStarted']
        
        # get trial id (path number)
        path_num = subject_session['trials'][0]['trialId']
        
        expt_path_start_time = sub_expt_data['TrialData'][path_num]['StartMachineTimeStamp'];
        expt_path_end_time = sub_expt_data['TrialData'][path_num]['EndMachineTimeStamp'];
        
        machine_time_stamp_diff = 0.0;
        if 'Single' not in condition and path_num != previous_path_num:
            expt_path_start_srvtime_mS = sub_expt_data['TrialData'][path_num]['StartServerTimeStamp'];
            machine_time_stamp_diff = expt_path_start_time - expt_path_start_srvtime_mS / 1000;
            previous_path_num = path_num;

        # amount of datapoints 
        Len_subses = len(subject_session['trials'][0]['dataPoints'])

        # for loop appending each data point rayCastHit Data
        # afterwards adding the timestamp to the dict 
        # passing if there is (1) no raycast hit and (2) if there is only one raycast hit
        for each in subject_session['trials'][0]['dataPoints']:

            # account for noHits 
            if each['rayCastHitsCombinedEyes'] == []:
                hitpoint_list.append(NoHit_dict)
            else:
                # append data point
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][0]) 
                # add path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = path_num
                
                # Exclude the data point if it lies outside of the times for a path
                if each['timeStampDataPointStart'] < expt_path_start_time or each['timeStampDataPointStart'] > expt_path_end_time:
                    hitpoint_list[idx]['PathNumber'] += 0; # Change this to some other high number to exclude
#                     print(str(idx) + " Data Pt. Start: " + str(each['timeStampDataPointStart']));
#                     print(str(idx) + " Data Pt. End: " + str(each['timeStampDataPointEnd']));
#                     print("Path Start: " + str(expt_path_start_time));
#                     print("Path End: " + str(expt_path_end_time));
#                     print('\n')

                # Add the server time stamp if dyadic
                if 'Single' not in condition:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']
                
                # Add HMD position
                hitpoint_list[idx]['hmdPosition'] = each['hmdPosition'];


            try: 
                # append data point of second raycast hit if it exists
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][1])
                # add Path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = hitpoint_list[idx-1]['PathNumber'];
                
                # Add the server time stamp if dyadic
                if 'Single' not in condition:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']
                
                

            except:
                pass

        # normalize the hitpoint dictionary to get dataframe
        hitpoints_df = pd.json_normalize(hitpoint_list)

        print("\tET: " + str(s) + " normalized")

        complete_hitpoints_df = complete_hitpoints_df.append(hitpoints_df)


    # --------- Saving each Session ---------

    # If you want to save each session separately, set 'Session_save_bool' to True
    if Session_save_bool == True:
        try:
            if len(subject_data) > 0:
                complete_hitpoints_df.to_csv(PROCESSED_DATA_PATH
                                             + str(subject)
                                             + "_CompleteSession"
                                             + "_" + condition
                                             + "_Hitpoints.csv")
                print("\t"
                      + str(subject)
                      + " session "
                      + "_" + condition
                      + " saved ")
            else: 
                print("\t"
                      + str(subject)
                      + " - Session "
                      + "_" + condition
                      + " is empty!")
        except:
            print("\tCould not save subject "
                  + str(subject)
                  + " session "
                  + "_" + condition
                  + "!")



    # fill the complete exploration dataframe with the separate session data (combining the sessions)
    complete_exploration_df = complete_exploration_df.append(complete_hitpoints_df)
        
    # --------- Saving the Exploration ---------
    
    # If you want to save the exploration file, set 'Exploration_save_bool' to True
    if Exploration_save_bool == True:
        # saving the complete exploration
        try:
            complete_exploration_df.to_csv(PROCESSED_DATA_PATH + str(subject) + "_" + condition + "_CompleteHitpoints.csv")
            print("\t" + str(subject) + " " + condition + " eyetracking data saved")
        except:
            print("\tCould not save subject " + str(subject) + " " + condition + " eyetracking data!")
    

print('Done')

Subject 1023 started - 1/1
	ET: 1 normalized
	ET: 2 normalized
	ET: 3 normalized
	ET: 4 normalized
	ET: 5 normalized
	ET: 6 normalized
	ET: 7 normalized
	ET: 8 normalized
	ET: 9 normalized
	ET: 10 normalized
	1023 Single eyetracking data saved
Done


In [72]:
Result_df = pd.read_csv(PROCESSED_DATA_PATH+'1023_Single_CompleteHitpoints.csv')

In [73]:
Result_df

,Unnamed: 0,PathNumber,timeStampDataPointStart,timeStampDataPointEnd,hitObjectColliderName,ordinalOfHit,BitMask,hitPointOnObject.x,hitPointOnObject.y,hitPointOnObject.z,hitObjectColliderBoundsCenter.x,hitObjectColliderBoundsCenter.y,hitObjectColliderBoundsCenter.z,hmdPosition.x,hmdPosition.y,hmdPosition.z
0,0,0.0,0.001984,0.007936,StartZoneA0,1.0,3.0,251.315308,-0.877272,170.721466,251.851456,-2.287690,171.080597,250.958450,-0.494329,170.39097595214844
1,1,0.0,0.001984,0.007936,road_base_network.004,2.0,3.0,252.663849,-2.399850,172.152802,39.257874,-4.408301,51.768860,NaN,NaN,NaN
2,2,0.0,0.008432,0.008432,StartZoneA0,1.0,3.0,251.315308,-0.877272,170.721466,251.851456,-2.287690,171.080597,250.958450,-0.494329,170.39097595214844
3,3,0.0,0.008432,0.008432,road_base_network.004,2.0,3.0,252.663849,-2.399850,172.152802,39.257874,-4.408301,51.768860,NaN,NaN,NaN
4,4,0.0,0.011904,0.014880,road_base_network.004,1.0,3.0,252.691284,-2.400710,172.184113,39.257874,-4.408301,51.768860,250.988846,-0.494738,170.42933654785156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248447,18489,9.0,90.206415,90.206415,terrain_C.001,2.0,3.0,-282.912537,-0.626187,-180.557816,-491.456299,18.698456,-597.844604,NaN,NaN,NaN
248448,18490,9.0,90.225760,90.228735,pavement_C.002,1.0,3.0,-282.881683,-0.488985,-180.404495,-217.756897,-2.905127,-288.595245,-282.382690,1.367091,-177.7785186767578
248449,18491,9.0,90.225760,90.228735,terrain_C.001,2.0,3.0,-282.919189,-0.627347,-180.598740,-491.456299,18.698456,-597.844604,NaN,NaN,NaN
248450,18492,9.0,90.228735,90.228735,pavement_C.002,1.0,3.0,-282.881683,-0.488985,-180.404495,-217.756897,-2.905127,-288.595245,-282.382690,1.367091,-177.7785186767578


In [19]:
print("data points excluded for 1023: ");
(Result_df['PathNumber'] > 20).value_counts()


data points excluded for 1023: 


False    248452
Name: PathNumber, dtype: int64

# Performance Analysis

In [56]:
pA_col_names = ['SubjectID', 'Condition', 'Dyadic?', 'Leader?', 'SessionID'];
for i in range(1,10):
    path_prefix = "P"+str(i);
    
    pA_col_names.append(path_prefix + ":TimeSec")
    pA_col_names.append(path_prefix + ":Success")
    pA_col_names.append(path_prefix + ":Time After Guide")
    pA_col_names.append(path_prefix + ":Distance Covered")

pA_col_names.append("#Success");


In [121]:
def get_covered_distances(subID, condition):
    # TODO: have to think about no-hits for this?
#     return [0] * 10;
    PROCESSED_DATA_PATH = './Results/' + condition + '/'
    try:
        filename = PROCESSED_DATA_PATH + str(subID) + '_' + condition +"_CompleteHitpoints.csv";
        data = pd.read_csv(filename);
    except:
        print('Could not read ' + filename);
        return[0] * 10;

    condense_col_names = ['PathNumber', 'xsqDiff', 'zsqDiff', 'eucDist']
    covered_dist_col_names = ['eucDist'];
    
    # Remove all second ordinal hits
    data = data.dropna(subset=['hmdPosition.x','hmdPosition.y','hmdPosition.z']);
    data = data.reset_index(drop=True)
    data['hmdPosition.z'] = data['hmdPosition.z'].astype(float);
    
    condense_df = pd.DataFrame(index=range(len(data)), columns=condense_col_names)
    cov_dist_df = pd.DataFrame(index=range(10), columns=covered_dist_col_names);
    
    condense_df.PathNumber = data.PathNumber
    condense_df.xsqDiff = data['hmdPosition.x'] - data['hmdPosition.x'].shift();
    condense_df.zsqDiff = data['hmdPosition.z'] - data['hmdPosition.z'].shift();
    
    condense_df.xsqDiff = condense_df.xsqDiff ** 2;
    condense_df.zsqDiff = condense_df.zsqDiff ** 2;
    
    condense_df.eucDist = (condense_df.xsqDiff + condense_df.zsqDiff) ** 0.5;
    
    cov_dist_df['eucDist'] = condense_df.groupby(['PathNumber'])['eucDist'].sum()
    
    return list(cov_dist_df.eucDist)

get_covered_distances('1023', 'Single')


[20.246065926035502,
 1319.0761375256886,
 300.24722235596226,
 172.22456297365136,
 1919.3239879759851,
 346.12679652061064,
 520.6331339390896,
 214.32876109412888,
 489.0953421457814,
 442.62558895940305]

In [129]:
perf_anal_df = pd.DataFrame(columns=pA_col_names);
for subject in subIDs:
    covered_distances = get_covered_distances(subject, condition);
    sub_expt_data = {}
    sub_expt_file_name = PROCESSED_DATA_PATH + str(subject) + "_"+condition+"Pathfinding_Final.json";
    try:
        with open(sub_expt_file_name, 'r') as fp:
            sub_expt_data = json.load(fp);
    except:
        raise Exception("Could not read experiment file for subject " + str(subject));
        
    sub_per_data = {'SubjectID' : sub_expt_data['ParticipantID'],
                    'Condition' : 'B' if sub_expt_data['PathsReversed'] else 'A',
                    'Dyadic?' : sub_expt_data['IsDyadic'],
                    'Leader?' : sub_expt_data['IsLeader']
                   };
    
    if sub_per_data['Dyadic?']: 
        sub_per_data['SessionID'] = dyadic_session_ids[sub_per_data['SubjectID']];
    else:
        sub_per_data['SessionID'] = 'NaN';
    
    path_time_limits = path_time_limits_B if sub_expt_data['PathsReversed'] else path_time_limits_A;
    num_success = 0;
    for i in range(1,10):
        current_trial_data = sub_expt_data['TrialData'][i];
        sub_per_data["P"+str(i)+":TimeSec"] = current_trial_data['EndMachineTimeStamp'] \
                                                - current_trial_data['StartMachineTimeStamp'];
        sub_per_data["P"+str(i)+":Success"] = not current_trial_data['GuideUsed'];
        if not sub_per_data["P"+str(i)+":Success"]: 
            sub_per_data["P"+str(i)+":Time After Guide"] = sub_per_data["P"+str(i)+":TimeSec"] - path_time_limits[i-1];
        else:
            sub_per_data["P"+str(i)+":Time After Guide"] = 0;
            num_success += 1;
        sub_per_data["P"+str(i)+":Distance Covered"] = covered_distances[i];
    
    sub_per_data['#Success'] = num_success;
    sub_df = pd.json_normalize(sub_per_data);
    
    perf_anal_df = perf_anal_df.append(sub_df);
    # TODO: Save to CSV

print('Analysis Complete');

Could not read ./Results/Single/1004_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1005_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1008_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1010_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1011_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1013_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1017_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1018_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1019_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1021_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1022_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1054_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1055_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1056_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1057_Single_CompleteHitpoints.csv
Could not 

In [130]:
perf_anal_df

,SubjectID,Condition,Dyadic?,Leader?,SessionID,P1:TimeSec,P1:Success,P1:Time After Guide,P1:Distance Covered,P2:TimeSec,...,P7:Distance Covered,P8:TimeSec,P8:Success,P8:Time After Guide,P8:Distance Covered,P9:TimeSec,P9:Success,P9:Time After Guide,P9:Distance Covered,#Success
0,1004,B,False,False,NaN,449.037666,False,117.037666,0,490.429310,...,0,390.436826,False,85.436826,0,113.507971,True,0,0,3
0,1005,A,False,False,NaN,158.458649,True,0.000000,0,114.378622,...,0,182.991541,True,0.000000,0,485.039520,False,153.03952,0,6
0,1008,B,False,False,NaN,89.310862,True,0.000000,0,112.268249,...,0,90.587071,True,0.000000,0,106.655506,True,0,0,8
0,1010,B,False,False,NaN,602.111310,False,270.111310,0,106.502529,...,0,162.011336,True,0.000000,0,196.642525,True,0,0,7
0,1011,B,False,False,NaN,102.925068,True,0.000000,0,177.524392,...,0,368.540978,False,63.540978,0,217.615655,True,0,0,7
0,1013,A,False,False,NaN,125.361833,True,0.000000,0,80.260620,...,0,136.801561,True,0.000000,0,438.497586,False,106.497586,0,8
0,1017,B,False,False,NaN,261.689323,True,0.000000,0,489.245452,...,0,432.588424,False,127.588424,0,289.393871,True,0,0,4
0,1018,B,False,False,NaN,165.254440,True,0.000000,0,377.181548,...,0,407.066795,False,102.066795,0,273.094684,True,0,0,7
0,1019,A,False,False,NaN,100.749390,True,0.000000,0,65.257159,...,0,114.624974,True,0.000000,0,470.427692,False,138.427692,0,7
0,1021,A,False,False,NaN,398.568231,False,98.568231,0,101.895389,...,0,118.187025,True,0.000000,0,447.441651,False,115.441651,0,6
